# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 24 2022 11:46AM,237856,18,HH6079,Hush Hush,Released
1,May 24 2022 11:45AM,237972,12,U0924,"Uniderm USA, Inc",Released
2,May 24 2022 11:37AM,237971,15,AL-000001084,"Alexso, Inc.",Released
3,May 24 2022 11:37AM,237970,10,0085712355,ISDIN Corporation,Released
4,May 24 2022 11:35AM,237969,10,8056978,Beach Products Inc,Released
5,May 24 2022 11:35AM,237967,10,PRF-29834,Bio-PRF,Released
6,May 24 2022 11:35AM,237967,10,PRF-29837,Bio-PRF,Released
7,May 24 2022 11:35AM,237967,10,PRF-29840,Bio-PRF,Released
8,May 24 2022 11:35AM,237967,10,PRF-29845,Bio-PRF,Released
9,May 24 2022 11:35AM,237967,10,PRF-29848,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,237968,Released,3
44,237969,Released,1
45,237970,Released,1
46,237971,Released,1
47,237972,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237968,NaN,NaN,3.0
237969,NaN,NaN,1.0
237970,NaN,NaN,1.0
237971,NaN,NaN,1.0
237972,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0.0,0.0,1.0
237701,0.0,0.0,1.0
237856,0.0,0.0,1.0
237861,0.0,10.0,13.0
237864,0.0,0.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237673,0,0,1
237701,0,0,1
237856,0,0,1
237861,0,10,13
237864,0,0,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,0,0,1
1,237701,0,0,1
2,237856,0,0,1
3,237861,0,10,13
4,237864,0,0,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237673,,,1
1,237701,,,1
2,237856,,,1
3,237861,,10,13
4,237864,,,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 24 2022 11:46AM,237856,18,Hush Hush
1,May 24 2022 11:45AM,237972,12,"Uniderm USA, Inc"
2,May 24 2022 11:37AM,237971,15,"Alexso, Inc."
3,May 24 2022 11:37AM,237970,10,ISDIN Corporation
4,May 24 2022 11:35AM,237969,10,Beach Products Inc
5,May 24 2022 11:35AM,237967,10,Bio-PRF
23,May 24 2022 11:34AM,237968,10,Emerginnova
26,May 24 2022 11:34AM,237966,15,"Laser Pharmaceuticals, LLC"
27,May 24 2022 11:27AM,237963,10,Eywa Pharma Inc.
55,May 24 2022 11:24AM,237965,18,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 24 2022 11:46AM,237856,18,Hush Hush,,,1
1,May 24 2022 11:45AM,237972,12,"Uniderm USA, Inc",,,1
2,May 24 2022 11:37AM,237971,15,"Alexso, Inc.",,,1
3,May 24 2022 11:37AM,237970,10,ISDIN Corporation,,,1
4,May 24 2022 11:35AM,237969,10,Beach Products Inc,,,1
5,May 24 2022 11:35AM,237967,10,Bio-PRF,,,18
6,May 24 2022 11:34AM,237968,10,Emerginnova,,,3
7,May 24 2022 11:34AM,237966,15,"Laser Pharmaceuticals, LLC",,,1
8,May 24 2022 11:27AM,237963,10,Eywa Pharma Inc.,,,28
9,May 24 2022 11:24AM,237965,18,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 11:46AM,237856,18,Hush Hush,1,,
1,May 24 2022 11:45AM,237972,12,"Uniderm USA, Inc",1,,
2,May 24 2022 11:37AM,237971,15,"Alexso, Inc.",1,,
3,May 24 2022 11:37AM,237970,10,ISDIN Corporation,1,,
4,May 24 2022 11:35AM,237969,10,Beach Products Inc,1,,
5,May 24 2022 11:35AM,237967,10,Bio-PRF,18,,
6,May 24 2022 11:34AM,237968,10,Emerginnova,3,,
7,May 24 2022 11:34AM,237966,15,"Laser Pharmaceuticals, LLC",1,,
8,May 24 2022 11:27AM,237963,10,Eywa Pharma Inc.,28,,
9,May 24 2022 11:24AM,237965,18,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 11:46AM,237856,18,Hush Hush,1,,
1,May 24 2022 11:45AM,237972,12,"Uniderm USA, Inc",1,,
2,May 24 2022 11:37AM,237971,15,"Alexso, Inc.",1,,
3,May 24 2022 11:37AM,237970,10,ISDIN Corporation,1,,
4,May 24 2022 11:35AM,237969,10,Beach Products Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 11:46AM,237856,18,Hush Hush,1.0,NaN,NaN
1,May 24 2022 11:45AM,237972,12,"Uniderm USA, Inc",1.0,NaN,NaN
2,May 24 2022 11:37AM,237971,15,"Alexso, Inc.",1.0,NaN,NaN
3,May 24 2022 11:37AM,237970,10,ISDIN Corporation,1.0,NaN,NaN
4,May 24 2022 11:35AM,237969,10,Beach Products Inc,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 24 2022 11:46AM,237856,18,Hush Hush,1.0,0.0,0.0
1,May 24 2022 11:45AM,237972,12,"Uniderm USA, Inc",1.0,0.0,0.0
2,May 24 2022 11:37AM,237971,15,"Alexso, Inc.",1.0,0.0,0.0
3,May 24 2022 11:37AM,237970,10,ISDIN Corporation,1.0,0.0,0.0
4,May 24 2022 11:35AM,237969,10,Beach Products Inc,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1903691,93.0,31.0,0.0
12,475855,1.0,1.0,0.0
15,1427670,36.0,29.0,16.0
16,951698,4.0,9.0,0.0
17,237924,1.0,0.0,0.0
18,1427058,8.0,17.0,0.0
19,951739,4.0,0.0,0.0
20,951517,39.0,36.0,0.0
21,237962,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1903691,93.0,31.0,0.0
1,12,475855,1.0,1.0,0.0
2,15,1427670,36.0,29.0,16.0
3,16,951698,4.0,9.0,0.0
4,17,237924,1.0,0.0,0.0
5,18,1427058,8.0,17.0,0.0
6,19,951739,4.0,0.0,0.0
7,20,951517,39.0,36.0,0.0
8,21,237962,0.0,1.0,0.0
9,22,475773,0.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,93.0,31.0,0.0
1,12,1.0,1.0,0.0
2,15,36.0,29.0,16.0
3,16,4.0,9.0,0.0
4,17,1.0,0.0,0.0
5,18,8.0,17.0,0.0
6,19,4.0,0.0,0.0
7,20,39.0,36.0,0.0
8,21,0.0,1.0,0.0
9,22,0.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,93.0
1,12,Released,1.0
2,15,Released,36.0
3,16,Released,4.0
4,17,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,17,18,19,20,21,22
Status,,,,,,,,,,
Completed,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,31.0,1.0,29.0,9.0,0.0,17.0,0.0,36.0,1.0,2.0
Released,93.0,1.0,36.0,4.0,1.0,8.0,4.0,39.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,31.0,1.0,29.0,9.0,0.0,17.0,0.0,36.0,1.0,2.0
2,Released,93.0,1.0,36.0,4.0,1.0,8.0,4.0,39.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,17,18,19,20,21,22
0,Completed,0.0,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,31.0,1.0,29.0,9.0,0.0,17.0,0.0,36.0,1.0,2.0
2,Released,93.0,1.0,36.0,4.0,1.0,8.0,4.0,39.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()